In [1]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
sns.set_style("whitegrid")

In [2]:
conn = sqlite3.connect("survey_data.db")

# Create a DataFrame from the "survey_responses" table
data = pd.read_sql_query("SELECT * FROM survey_responses", conn)

conn.close()

In [3]:
data

,timestamp,gender,school_year,program_faculty,residence,most_used_device,most_time_spent_on,daily_online_time_on_campus,most_active_time_on_campus,primary_internet_providers,...,mtn_experience_rating,airtel_experience_rating,glo_experience_rating,nine_mobile_experience_rating,afit_cict_experience_rating,poor_service_coping_methods,academic_impact_of_poor_internet,financial_impact_of_poor_internet,planning_to_switch_provider,suggestions_for_improvement
0,2025-02-13 01:39:04,Male,400L,Faculty of Ground and Communications Engineering,School Hostel,Laptop,"Video Streaming, Socials, Messaging",5 - 8 hours,After Dark (12am - 5am),"Airtel, AFIT CICT",...,2.0,3.0,0.0,0.0,3.0,"Using multiple providers, Changing location wh...",Yes,Yes,Maybe,None
1,2025-02-13 01:44:37,Male,400L,Faculty of Air Engineering,Barkallahu,Phone,Socials,3 -5 hours,Afternoon,"MTN, Airtel, GLO",...,1.0,1.0,3.0,0.0,2.0,Using multiple providers,Yes,No,Yes,None
2,2025-02-13 01:52:48,Male,400L,Faculty of Ground and Communications Engineering,Barkallahu,Phone,"Video Streaming, Socials, Productivity/Work",Less than 1 hour,Evening,MTN,...,2.0,0.0,0.0,0.0,2.0,"Changing location when in need of internet, Re...",No,No,No,None
3,2025-02-13 01:58:11,Male,400L,Faculty of Ground and Communications Engineering,Barkallahu,Phone,"Socials, Messaging, Productivity/Work",Less than 1 hour,Afternoon,"MTN, Airtel, GLO",...,1.0,2.0,3.0,0.0,2.0,"Using multiple providers, Changing location wh...",No,Yes,Maybe,"Disable the ""scramblers"" if there are or find ..."
4,2025-02-13 02:01:31,Male,400L,Faculty of Ground and Communications Engineering,Barkallahu,Phone,"Video Streaming, Messaging, Productivity/Work",3 -5 hours,Afternoon,"MTN, Airtel, AFIT CICT",...,1.0,1.0,0.0,0.0,1.0,Using multiple providers,Yes,Yes,No,Better WiFi and service provider connectivity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,2025-03-04 23:27:34,Female,400L,Faculty of Science,School Hostel,Phone,Socials,1 - 3 hours,Night,"MTN, Airtel",...,2.0,3.0,0.0,0.0,2.0,Changing location when in need of internet,No,No,Maybe,None
323,2025-03-05 12:27:37,Male,100L,Faculty of Ground and Communications Engineering,School Hostel,Phone,"Socials, Messaging, Productivity/Work",3 -5 hours,Afternoon,"MTN, Airtel",...,1.0,1.0,0.0,0.0,0.0,"Using multiple providers, Changing location wh...",No,Yes,No,None
324,2025-03-09 23:21:48,Male,400L,Faculty of Science,School Hostel,Phone,"Movie & TV downloads, Messaging, Readable content",1 - 3 hours,Afternoon,"MTN, GLO, AFIT CICT",...,1.0,2.0,2.0,3.0,2.0,"Changing location when in need of internet, Re...",Yes,No,Yes,None
325,2025-03-15 11:36:58,Male,300L,Faculty of Computing,School Hostel,Phone,"Messaging, Productivity/Work, Readable content",3 -5 hours,After Dark (12am - 5am),"MTN, Airtel, AFIT CICT",...,3.0,3.0,0.0,0.0,4.0,"Using multiple providers, Staying awake midnig...",Yes,Yes,No,AFIT should provide more Wifi with higher freq...


## Data Cleaning and Normalization

In [4]:
data["timestamp"] = pd.to_datetime(data["timestamp"])

def clean_data(data):
    # Replace missing values with "Graduate" in column: 'school_year'
    data = data.fillna({'school_year': "Graduate"})
    # Replace missing values with "No Faculty" in column: 'program_faculty'
    data = data.fillna({'program_faculty': "No Faculty"})
    # Remove leading and trailing whitespace in column: 'most_used_device'
    data['most_used_device'] = data['most_used_device'].str.strip()
    # Replace all instances of "phone and laptop" with "Phone" in column: 'most_used_device'
    data.loc[data['most_used_device'].str.lower() == "phone and laptop".lower(), 'most_used_device'] = "Phone"
    # Replace missing values with "None" in column: 'most_time_spent_on'
    data = data.fillna({'most_time_spent_on': "None"})
    # Replace missing values with "None" in column: 'suggestions_for_improvement'
    data = data.fillna({'suggestions_for_improvement': "None"})
    return data

data = clean_data(data.copy())
data.head()

,timestamp,gender,school_year,program_faculty,residence,most_used_device,most_time_spent_on,daily_online_time_on_campus,most_active_time_on_campus,primary_internet_providers,...,mtn_experience_rating,airtel_experience_rating,glo_experience_rating,nine_mobile_experience_rating,afit_cict_experience_rating,poor_service_coping_methods,academic_impact_of_poor_internet,financial_impact_of_poor_internet,planning_to_switch_provider,suggestions_for_improvement
0,2025-02-13 01:39:04,Male,400L,Faculty of Ground and Communications Engineering,School Hostel,Laptop,"Video Streaming, Socials, Messaging",5 - 8 hours,After Dark (12am - 5am),"Airtel, AFIT CICT",...,2.0,3.0,0.0,0.0,3.0,"Using multiple providers, Changing location wh...",Yes,Yes,Maybe,None
1,2025-02-13 01:44:37,Male,400L,Faculty of Air Engineering,Barkallahu,Phone,Socials,3 -5 hours,Afternoon,"MTN, Airtel, GLO",...,1.0,1.0,3.0,0.0,2.0,Using multiple providers,Yes,No,Yes,None
2,2025-02-13 01:52:48,Male,400L,Faculty of Ground and Communications Engineering,Barkallahu,Phone,"Video Streaming, Socials, Productivity/Work",Less than 1 hour,Evening,MTN,...,2.0,0.0,0.0,0.0,2.0,"Changing location when in need of internet, Re...",No,No,No,None
3,2025-02-13 01:58:11,Male,400L,Faculty of Ground and Communications Engineering,Barkallahu,Phone,"Socials, Messaging, Productivity/Work",Less than 1 hour,Afternoon,"MTN, Airtel, GLO",...,1.0,2.0,3.0,0.0,2.0,"Using multiple providers, Changing location wh...",No,Yes,Maybe,"Disable the ""scramblers"" if there are or find ..."
4,2025-02-13 02:01:31,Male,400L,Faculty of Ground and Communications Engineering,Barkallahu,Phone,"Video Streaming, Messaging, Productivity/Work",3 -5 hours,Afternoon,"MTN, Airtel, AFIT CICT",...,1.0,1.0,0.0,0.0,1.0,Using multiple providers,Yes,Yes,No,Better WiFi and service provider connectivity


In [ ]:
# exploded = data["most_time_spent_on"].str.split(', ').explode()
# exploded.value_counts()

most_time_spent_on
Productivity/Work                          190
Socials                                    179
Messaging                                  127
Readable content                           120
Movie & TV downloads                        75
Audio streaming or downloads                65
Video Streaming                             64
Gaming                                      49
                                             2
ALL THE ABOVE                                1
Research                                     1
mobile money transfer                        1
YouTube                                      1
The network is bad can't use it at all       1
Research with search engines                 1
Online courses                               1
Manga downloading                            1
YouTube content related to my course         1
None                                         1
Name: count, dtype: int64